# ETL Project
###
### Team 6 Project Members:  Nisha Saphota, Daniel Eddie, Jon Simpson, Rob Gauer
### Date Due:  Tuesday June 9, 2020

In [1]:
# Dependencies and Setup
import pandas as pd

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

## Store CSV into DataFrame

In [2]:
# Create dataframe from Kaggle dataset
kaggle_csv_file = "./Resources/spotify_60k_kaggle_master_song.csv"
album_data_df = pd.read_csv(kaggle_csv_file)

# display the contents of the data frame 
album_data_df.head()

,Release Date,Artist,Album,Track ID,Track,Popularity,Danceability,Energy,Key,Key Val,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature
0,2013,The Devil Wears Prada,0.345833333,56PpndWxF2IGNQeBATYEEl,Rumors,31,0.455,0.972,A,9,-3.025,1,0.1870,0.000193,0.000229,0.387,0.102,133.034,4
1,2011,BeyoncÃ©,4,1uXbwHHfgsXcUKfSZw5ZJ0,Run the World (Girls),73,0.733,0.899,C,0,-4.237,1,0.1430,0.004960,0.000047,0.372,0.760,127.086,4
2,1998,Lenny Kravitz,5,6vUoqsJ0uVBgSKaUAUEQYC,Live,39,0.632,0.926,G,7,-5.954,1,0.1270,0.008900,0.466000,0.367,0.777,168.284,4
3,1998,Lenny Kravitz,5,0KF7XWr4IxZsmD1DnSkDwh,Supersoulfighter,30,0.747,0.868,B,11,-6.360,0,0.0613,0.002470,0.174000,0.494,0.485,110.950,4
4,2000,Lenny Kravitz,5,2zee8Zcesqwnnwliw2Jy8M,I Belong To You,57,0.690,0.664,F,5,-7.715,0,0.0542,0.000447,0.034300,0.073,0.630,87.287,4


In [3]:
# Create dataframe from Zenodo dataset
zenodo_csv_file = "./Resources/spotify_zenodo_output_data.csv"
album_reviews_all_columns_df = pd.read_csv(zenodo_csv_file)

# display the contents of the data frame 
album_reviews_all_columns_df.head()

,artist,album,reviewauthor,score,releaseyear,reviewdate,recordlabel,genre,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Studio 1,Studio 1,Andy Battaglia,8.5,2009.0,February 18 2009,Studio,Electronic,0.511917,0.499667,5.250000,-5.626583,0.031983,0.724917,0.024493,0.165367,0.555083,101.395167
1,John Fahey,The Great Santa Barbara Oil Slick,Mark Richardson,8.2,2005.0,February 13 2005,Water,Folk/Country,0.369765,0.325412,4.470588,-19.153824,0.148624,0.647053,0.559133,0.527782,0.179465,107.622647
2,Reigning Sound,Too Much Guitar,Stephen M. Deusner,8.3,2004.0,August 19 2004,In the Red,Electronic,0.253943,0.912857,4.428571,-1.089500,0.055500,0.000253,0.751214,0.199071,0.552786,133.895500
3,The Red Thread,After the Last,Chris Dahlen,7.3,2003.0,July 17 2003,Badman,Rock,0.425400,0.433474,5.700000,-12.871000,0.028260,0.310325,0.224137,0.125150,0.451400,104.354200
4,Mac Miller,Swimming,Evan Rytlewski,7.5,2018.0,August 3 2018,Warner Bros.,Rap,0.624846,0.438154,4.153846,-9.456077,0.170246,0.652462,0.012819,0.121131,0.281138,122.121308


## Review DataFrames and Prepare for Merge

In [4]:
# DataFrame 'album_data_df' change 'Album' and 'Artist' columns to all uppercase.
album_data_df['Album']=album_data_df['Album'].str.upper()
album_data_df['Artist']=album_data_df['Artist'].str.upper()

# display the contents of the data frame 
album_data_df.head()

,Release Date,Artist,Album,Track ID,Track,Popularity,Danceability,Energy,Key,Key Val,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature
0,2013,THE DEVIL WEARS PRADA,0.345833333,56PpndWxF2IGNQeBATYEEl,Rumors,31,0.455,0.972,A,9,-3.025,1,0.1870,0.000193,0.000229,0.387,0.102,133.034,4
1,2011,BEYONCÃ©,4,1uXbwHHfgsXcUKfSZw5ZJ0,Run the World (Girls),73,0.733,0.899,C,0,-4.237,1,0.1430,0.004960,0.000047,0.372,0.760,127.086,4
2,1998,LENNY KRAVITZ,5,6vUoqsJ0uVBgSKaUAUEQYC,Live,39,0.632,0.926,G,7,-5.954,1,0.1270,0.008900,0.466000,0.367,0.777,168.284,4
3,1998,LENNY KRAVITZ,5,0KF7XWr4IxZsmD1DnSkDwh,Supersoulfighter,30,0.747,0.868,B,11,-6.360,0,0.0613,0.002470,0.174000,0.494,0.485,110.950,4
4,2000,LENNY KRAVITZ,5,2zee8Zcesqwnnwliw2Jy8M,I Belong To You,57,0.690,0.664,F,5,-7.715,0,0.0542,0.000447,0.034300,0.073,0.630,87.287,4


In [5]:
# DataFrame 'album_reviews_all_columns_df' change 'Album' and 'Artist' columns to all uppercase.
album_reviews_all_columns_df['album']=album_reviews_all_columns_df['album'].str.upper()
album_reviews_all_columns_df['artist']=album_reviews_all_columns_df['artist'].str.upper()

# display the contents of the data frame 
album_reviews_all_columns_df.head()

,artist,album,reviewauthor,score,releaseyear,reviewdate,recordlabel,genre,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,STUDIO 1,STUDIO 1,Andy Battaglia,8.5,2009.0,February 18 2009,Studio,Electronic,0.511917,0.499667,5.250000,-5.626583,0.031983,0.724917,0.024493,0.165367,0.555083,101.395167
1,JOHN FAHEY,THE GREAT SANTA BARBARA OIL SLICK,Mark Richardson,8.2,2005.0,February 13 2005,Water,Folk/Country,0.369765,0.325412,4.470588,-19.153824,0.148624,0.647053,0.559133,0.527782,0.179465,107.622647
2,REIGNING SOUND,TOO MUCH GUITAR,Stephen M. Deusner,8.3,2004.0,August 19 2004,In the Red,Electronic,0.253943,0.912857,4.428571,-1.089500,0.055500,0.000253,0.751214,0.199071,0.552786,133.895500
3,THE RED THREAD,AFTER THE LAST,Chris Dahlen,7.3,2003.0,July 17 2003,Badman,Rock,0.425400,0.433474,5.700000,-12.871000,0.028260,0.310325,0.224137,0.125150,0.451400,104.354200
4,MAC MILLER,SWIMMING,Evan Rytlewski,7.5,2018.0,August 3 2018,Warner Bros.,Rap,0.624846,0.438154,4.153846,-9.456077,0.170246,0.652462,0.012819,0.121131,0.281138,122.121308


In [6]:
# Create a filtered dataframe from specific columns
# Revise dataframe from Zenodo dataset to only include the data columns of 'album', 'score', 'artist', and reviewauthor'.
album_reviews_data_df=album_reviews_all_columns_df[['album','score','artist','reviewauthor']]

# display the contents of the data frame 
album_reviews_data_df.head()

,album,score,artist,reviewauthor
0,STUDIO 1,8.5,STUDIO 1,Andy Battaglia
1,THE GREAT SANTA BARBARA OIL SLICK,8.2,JOHN FAHEY,Mark Richardson
2,TOO MUCH GUITAR,8.3,REIGNING SOUND,Stephen M. Deusner
3,AFTER THE LAST,7.3,THE RED THREAD,Chris Dahlen
4,SWIMMING,7.5,MAC MILLER,Evan Rytlewski


In [7]:
# Review data counts
album_data_df.count()

Release Date        61044
Artist              61044
Album               61044
Track ID            61044
Track               61044
Popularity          61044
Danceability        61044
Energy              61044
Key                 61044
Key Val             61044
Loudness            61044
Mode                61044
Speechiness         61044
Acousticness        61044
Instrumentalness    61044
Liveness            61044
Valence             61044
Tempo               61044
Time Signature      61044
dtype: int64

In [8]:
# Review data counts
album_reviews_data_df.count()

album           18403
score           18403
artist          18402
reviewauthor    18403
dtype: int64

In [9]:
# Collecting a list of all columns within the DataFrame
album_data_df.columns

#for col in album_data_df:
#    print(col)

Index(['Release Date', 'Artist', 'Album', 'Track ID', 'Track', 'Popularity',
       'Danceability', 'Energy', 'Key', 'Key Val', 'Loudness', 'Mode',
       'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness',
       'Valence', 'Tempo', 'Time Signature'],
      dtype='object')

In [10]:
# Collecting a list of all columns within the DataFrame
album_reviews_data_df.columns
#for columns in album_reviews_data_df:
#    print(columns)

Index(['album', 'score', 'artist', 'reviewauthor'], dtype='object')

In [11]:
# Rename columns headers in dataframe 
album_reviews_data_df=album_reviews_data_df.rename(columns={"artist":"Artist","album":"Album","reviewauthor":"Review_Author","score":"Score","releaseyear":"Release_Year","reviewdate":"Review_Date","recordlabel":"Record_Label","genre":"Genre","danceability":"Danceability","energy":"Energy","key":"Key","loudness":"Loudness","speechiness":"Speechiness","acousticness":"Acousticness","instrumentalness":"Instrumentalness","liveness":"Liveness","valence":"Valence","tempo":"Tempo"})

# display the contents of the data frame 
album_reviews_data_df.head()

,Album,Score,Artist,Review_Author
0,STUDIO 1,8.5,STUDIO 1,Andy Battaglia
1,THE GREAT SANTA BARBARA OIL SLICK,8.2,JOHN FAHEY,Mark Richardson
2,TOO MUCH GUITAR,8.3,REIGNING SOUND,Stephen M. Deusner
3,AFTER THE LAST,7.3,THE RED THREAD,Chris Dahlen
4,SWIMMING,7.5,MAC MILLER,Evan Rytlewski


In [12]:
# Display data information
album_reviews_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18403 entries, 0 to 18402
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Album          18403 non-null  object 
 1   Score          18403 non-null  float64
 2   Artist         18402 non-null  object 
 3   Review_Author  18403 non-null  object 
dtypes: float64(1), object(3)
memory usage: 575.2+ KB


## Merge the two DataFrames into a single dataset

In [13]:
# Create the Final Dataframe - presentation 
# Combine the two datasets (DataFrames) into a single dataset (DataFrame)
#merge_datasets_df=pd.merge(album_reviews_data_df,album_data_df,on="Album")
merge_datasets_df = pd.merge(album_reviews_data_df, album_data_df, on=['Album', 'Artist'])

# display the contents of the data frame 
merge_datasets_df

,Album,Score,Artist,Review_Author,Release Date,Track ID,Track,Popularity,Danceability,Energy,...,Key Val,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature
0,SWIMMING,7.5,MAC MILLER,Evan Rytlewski,2018,39NDBdU5Xkm5pCFGa5kZtI,Ladders,73,0.802,0.463,...,8,-8.379,1,0.1620,0.23600,0.005310,0.1050,0.291,103.961,4
1,SWIMMING,7.5,MAC MILLER,Evan Rytlewski,2018,01z2fBGB8Hl3Jd3zXe4IXR,Come Back to Earth,72,0.272,0.238,...,2,-12.148,0,0.0349,0.89000,0.009250,0.2750,0.119,83.507,4
2,SWIMMING,7.5,MAC MILLER,Evan Rytlewski,2018,5p7GiBZNL1afJJDUrOA6C8,Hurt Feelings,71,0.690,0.462,...,8,-8.054,1,0.1300,0.34300,0.002180,0.1130,0.325,157.655,4
3,SWIMMING,7.5,MAC MILLER,Evan Rytlewski,2018,2dgrYdgguVZKeCsrVb9XEs,What's the Use?,70,0.759,0.492,...,1,-10.338,0,0.1200,0.73600,0.009890,0.1070,0.561,104.974,4
4,SWIMMING,7.5,MAC MILLER,Evan Rytlewski,2018,0Xcy81PsJCOO6mhLZaQyQ4,Perfecto,64,0.599,0.317,...,1,-11.212,1,0.1740,0.74800,0.000000,0.1110,0.203,148.090,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4372,BACK IN BLACK,8.8,AC/DC,Steve Kandell,1980,5XeMsYb1rWxQL1bUUaZajn,Let Me Put My Love Into You,52,0.400,0.902,...,9,-4.385,1,0.0523,0.11500,0.000116,0.1390,0.632,105.659,4
4373,BACK IN BLACK,8.8,AC/DC,Steve Kandell,1980,2SiXAy7TuUkycRVbbWDEpo,You Shook Me All Night Long,77,0.532,0.767,...,7,-5.509,1,0.0574,0.00287,0.000513,0.3900,0.755,127.361,4
4374,BACK IN BLACK,8.8,AC/DC,Steve Kandell,1980,2MCUmp0I98gsJBmey7DfZS,Shake a Leg,50,0.417,0.942,...,4,-4.225,1,0.0852,0.02340,0.008570,0.3170,0.618,152.346,4
4375,BACK IN BLACK,8.8,AC/DC,Steve Kandell,1980,6J17MkMmuzBiIOjRH6MOBZ,Rock and Roll Ain't Noise Pollution,60,0.576,0.659,...,9,-5.617,1,0.0383,0.04100,0.000000,0.0587,0.717,94.853,4


In [14]:
# Review data counts of merged DataFrames
merge_datasets_df.count()

Album               4377
Score               4377
Artist              4377
Review_Author       4377
Release Date        4377
Track ID            4377
Track               4377
Popularity          4377
Danceability        4377
Energy              4377
Key                 4377
Key Val             4377
Loudness            4377
Mode                4377
Speechiness         4377
Acousticness        4377
Instrumentalness    4377
Liveness            4377
Valence             4377
Tempo               4377
Time Signature      4377
dtype: int64

# Transform - Review/clean the combined DataFrame

In [15]:
#Save DataFrame to a csv file: /Output_Data/merge_datasets_df_pre_clean.csv
merge_datasets_df.to_csv('Output_Data/1_merge_datasets_df_pre_clean.csv', encoding="utf-8", index="true",header="true")

In [16]:
# Interrogate dataset - Quantity of Review Authors.
len(merge_datasets_df["Review_Author"].unique())

173

In [17]:
# Interrogate dataset- Quantity of Track IDs.
len(merge_datasets_df["Track ID"].unique())

4352

In [18]:
# Interrogate dataset- Quantity of Artists.
len(merge_datasets_df["Artist"].unique())

323

In [19]:
# Interrogate dataset- Quantity of Albums.
len(merge_datasets_df["Album"].unique())

504

In [20]:
# Interrogate dataset- Quantity of Tracks.
len(merge_datasets_df["Track"].unique())

4262

In [21]:
# Interrogate dataset - Boolean test for duplicates.
merge_datasets_df.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
4372    False
4373    False
4374    False
4375    False
4376    False
Length: 4377, dtype: bool

In [22]:
# Interrogate dataset - Identify duplicate 'Track ID's - If they exist.
merge_datasets_df.loc[merge_datasets_df.duplicated("Track ID")== True]

,Album,Score,Artist,Review_Author,Release Date,Track ID,Track,Popularity,Danceability,Energy,...,Key Val,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature
14,AMERICAN FOOTBALL,7.7,AMERICAN FOOTBALL,Ian Cohen,1999,51R5mPcJjOnfv9lKY1u5sW,Never Meant,57,0.436,0.819,...,0,-6.710,1,0.0360,0.017100,0.008820,0.1170,0.405,144.260,4
448,1999,10.0,PRINCE,Paul A. Thompson,1982,2H7PHVdQ3mXqEHXcvclTB0,1999,67,0.866,0.730,...,5,-8.201,1,0.0767,0.137000,0.000000,0.0843,0.625,118.523,4
449,1999,10.0,PRINCE,Paul A. Thompson,1982,4iozhXt27eMl39W5z7R8H6,Little Red Corvette,53,0.748,0.746,...,1,-9.721,1,0.1110,0.281000,0.000033,0.6320,0.709,123.144,4
450,1999,10.0,PRINCE,Paul A. Thompson,1982,6hazdpTPlt5W2BTCGYKBoj,Delirious,47,0.920,0.402,...,0,-16.656,1,0.0549,0.058600,0.000434,0.0234,0.957,99.910,4
451,1999,10.0,PRINCE,Paul A. Thompson,1982,31ink8UgWSYUXz0hPasoif,Let's Pretend We're Married,32,0.790,0.455,...,10,-15.194,0,0.1380,0.003180,0.000104,0.4560,0.819,184.761,4
452,1999,10.0,PRINCE,Paul A. Thompson,1982,1YpgRBDgD8ed7eb8i053Qt,D.M.S.R.,35,0.970,0.499,...,11,-8.447,1,0.1640,0.066900,0.000010,0.1800,0.740,109.567,4
453,1999,10.0,PRINCE,Paul A. Thompson,1982,1MpPRotMSsZhI5eevP2qeO,Automatic,30,0.830,0.422,...,8,-16.141,1,0.0750,0.040300,0.010400,0.0732,0.902,124.907,4
454,1999,10.0,PRINCE,Paul A. Thompson,1982,2hABMU63xtaIYChN6eYlEb,Something in the Water (Does Not Compute),37,0.614,0.415,...,3,-18.296,0,0.0636,0.095000,0.000169,0.1500,0.725,169.002,4
455,1999,10.0,PRINCE,Paul A. Thompson,1982,2dBRAH9J3fL24AJkYmjZno,Free,27,0.558,0.459,...,2,-11.218,1,0.0630,0.376000,0.001810,0.1010,0.520,65.393,4
456,1999,10.0,PRINCE,Paul A. Thompson,1982,3olcbtUJV3xdHIfFjy8owe,Lady Cab Driver,40,0.855,0.502,...,0,-15.226,1,0.0947,0.044200,0.089800,0.6500,0.749,122.645,4


In [23]:
# Found 25 rows of duplicate 'Track ID' records.
# Clean the data by dropping 'Track ID' duplicates and setting the index
merge_datasets_df.drop_duplicates('Track ID', inplace=True)

In [33]:
# Interrogate dataset- Quantity of Track IDs updated.
len(merge_datasets_df["Track ID"].unique())

4352

In [34]:
# Interrogate dataset - SORT by 'Track ID','Track','Artist','Album','Score'.

# Sort by multiple columns
merge_datasets_df.sort_values(by=['Track ID','Track','Artist','Album','Score'], inplace=True)

# display the contents of the data frame 
merge_datasets_df.head()

,Album,Score,Artist,Review_Author,Release Date,Track ID,Track,Popularity,Danceability,Energy,...,Key Val,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature
557,BECAUSE THE INTERNET,5.8,CHILDISH GAMBINO,Craig Jenkins,2013,00GOPLxW4PGQuUYdPJh8K1,IV. Sweatpants,71,0.710,0.4700,...,1,-9.330,0,0.1420,0.1540,0.000000,0.643,0.581,80.027,4
2766,TRACY CHAPMAN,9.4,TRACY CHAPMAN,Ann-Derrick Gaillot,1988,00OMaIm4VEsUcXad5Pf1Qe,Behind the Wall,46,0.668,0.0505,...,5,-18.843,1,0.3580,0.8070,0.000000,0.107,0.598,148.649,5
932,WHITE LIGHT/WHITE HEAT,10.0,THE VELVET UNDERGROUND,Douglas Wolk,1968,00bScuvzFpyRMGWlgfu5sp,Lady Godiva's Operation,34,0.507,0.5300,...,4,-13.059,0,0.0368,0.7540,0.000599,0.143,0.773,114.412,4
1594,SOMETHING/ANYTHING?,9.0,TODD RUNDGREN,Sam Sodomsky,1972,00f5cJ88kjiyCLJebRHgAY,It Wouldn't Have Made Any Difference,34,0.560,0.3920,...,9,-13.100,1,0.0296,0.2840,0.000000,0.074,0.652,172.327,4
3379,GOOD NATURE,7.3,TURNOVER,Ian Cohen,2017,00gLuZIDF5bxUEO0RkPx1m,All That It Ever Was,32,0.614,0.6760,...,9,-7.902,1,0.0270,0.0442,0.019900,0.112,0.722,130.986,4


In [35]:
# Interrogate dataset - SORT by 'Score','Artist','Album','Track ID','Track'.

# Sort by multiple columns
merge_datasets_df.sort_values(by=['Score','Artist','Album','Track ID','Track'], ascending=False, inplace=True)

# display the contents of the data frame 
merge_datasets_df.head()

,Album,Score,Artist,Review_Author,Release Date,Track ID,Track,Popularity,Danceability,Energy,...,Key Val,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature
840,PINK FLAG,10.0,WIRE,Joe Tangari,1977,6tPUscoU7EpieQLHBBWd0q,106 Beats That - 2006 Remastered Version,28,0.368,0.878,...,6,-7.417,0,0.0373,0.000787,0.617000,0.2150,0.963,160.314,4
833,PINK FLAG,10.0,WIRE,Joe Tangari,1977,6ZzgkNscFRe9PiryaeUVwo,Start to Move - 2006 Remastered Version,35,0.617,0.825,...,11,-7.147,1,0.0591,0.044400,0.000003,0.0398,0.952,103.225,4
839,PINK FLAG,10.0,WIRE,Joe Tangari,1977,6X7CDzfm1Nq7qZwmRaSUWS,Straight Line - 2006 Remastered Version,28,0.455,0.937,...,2,-5.845,1,0.0472,0.576000,0.000071,0.2100,0.685,116.112,4
837,PINK FLAG,10.0,WIRE,Joe Tangari,1977,6VvpBt95FoPF01YBqv0vV1,Pink Flag - 2006 Remastered Version,31,0.451,0.844,...,9,-9.692,1,0.0672,0.261000,0.850000,0.1070,0.201,122.168,4
846,PINK FLAG,10.0,WIRE,Joe Tangari,1977,6Slsb9yR7U2gvgw4GEGkbV,Champs - 2006 Remastered Version,29,0.353,0.811,...,9,-7.224,1,0.0403,0.159000,0.000655,0.0579,0.886,169.016,4


In [44]:
# Interrogate dataset - Resort to ascending based on album

# Sort by multiple columns
merge_datasets_df.sort_index(inplace=True)

In [45]:
# verify counts - validate we now have a clean dataset
merge_datasets_df.count()

Album               4352
Score               4352
Artist              4352
Review_Author       4352
Release Date        4352
Track ID            4352
Track               4352
Popularity          4352
Danceability        4352
Energy              4352
Key                 4352
Key Val             4352
Loudness            4352
Mode                4352
Speechiness         4352
Acousticness        4352
Instrumentalness    4352
Liveness            4352
Valence             4352
Tempo               4352
Time Signature      4352
dtype: int64

In [50]:
# Rename the DataFrame and Display the contents of the final clean DataFrame 
album_reviews_df=merge_datasets_df
album_reviews_df.head()

,Album,Score,Artist,Review_Author,Release Date,Track ID,Track,Popularity,Danceability,Energy,...,Key Val,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature
0,SWIMMING,7.5,MAC MILLER,Evan Rytlewski,2018,39NDBdU5Xkm5pCFGa5kZtI,Ladders,73,0.802,0.463,...,8,-8.379,1,0.1620,0.236,0.00531,0.105,0.291,103.961,4
1,SWIMMING,7.5,MAC MILLER,Evan Rytlewski,2018,01z2fBGB8Hl3Jd3zXe4IXR,Come Back to Earth,72,0.272,0.238,...,2,-12.148,0,0.0349,0.890,0.00925,0.275,0.119,83.507,4
2,SWIMMING,7.5,MAC MILLER,Evan Rytlewski,2018,5p7GiBZNL1afJJDUrOA6C8,Hurt Feelings,71,0.690,0.462,...,8,-8.054,1,0.1300,0.343,0.00218,0.113,0.325,157.655,4
3,SWIMMING,7.5,MAC MILLER,Evan Rytlewski,2018,2dgrYdgguVZKeCsrVb9XEs,What's the Use?,70,0.759,0.492,...,1,-10.338,0,0.1200,0.736,0.00989,0.107,0.561,104.974,4
4,SWIMMING,7.5,MAC MILLER,Evan Rytlewski,2018,0Xcy81PsJCOO6mhLZaQyQ4,Perfecto,64,0.599,0.317,...,1,-11.212,1,0.1740,0.748,0.00000,0.111,0.203,148.090,4


In [52]:
#Save transformed DataFrame to a new csv file: /Output_Data/album_reviews_dataframe.csv
album_reviews_df.to_csv('Output_Data/3_album_reviews_dataframe.csv', encoding="utf-8", index="true",header="true")

# Create final DataFrame for database creation

In [ ]:
# Create new clean datafrome
album_reviews_df=clean_merge_datasets_df

# display the contents of the data frame 
album_reviews_df.head()

In [ ]:
# Display data information for review and setup of database
album_reviews_df.info()

In [ ]:
# Display data types and review
album_reviews_df.dtypes

## Connect to local database 

In [ ]:
# Launch Postgres PG Admin4 and create PostgreSQL database called 'album_db'. Then run this command string.
# connection_string = "<insert user name>:<insert password>@localhost:5432/album_db"
connection_string = "postgres:postgres@localhost:5432/album_db"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
# Confirm tables (no data exists at this time...load data into db)
engine.table_names()

## Load DataFrame into database (postgreSQL)

In [ ]:
# Use pandas to load csv converted DataFrame into database
album_reviews_df.to_sql(name='Album_db', con=engine, if_exists='', index=True)

In [ ]:
# Confirm tables (data exists at this time...)
engine.table_names()

# Confirm data has been added by querying the customer_name table
# NOTE: can also check using pgAdmin

In [ ]:
album_db.read_sql_query('select * from Album', con=engine).head()

## Load DataFrame into database (SQLite)

In [ ]:
## Import SQL Alchemy
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy

## SQL Alchemy Configuration Parameters 
# automap_base required for Reflection...
from sqlalchemy.ext.automap import automap_base  

from sqlalchemy.orm import Session
#from sqlalchemy import create_engine, func

# inspect required for Inspection...
from sqlalchemy import create_engine, func  

In [ ]:
# Path to sqlite
album_database_path = "./Output_Data/album_reviews.sqlite"

# Create an engine that can talk to the database
#engine = create_engine("sqlite:///Resources/hawaii.sqlite")
engine = create_engine(f"sqlite:///{album_database_path}")

# To enable Pandas and create a DataFrame
connect = engine.connect()

In [ ]:
# Confirm tables (data exists at this time...)
engine.table_names()

## -- EOF -- ## 